# Dados

In [ ]:
import numpy as np
import pandas as pd
data=pd.read_csv("speedDating_trab.csv", sep=",")

#retirando a primeira coluna undefined
data=pd.DataFrame(data=data.iloc[:,1:14].values,columns=data.columns[1:14])

print(data)


        id  partner   age  age_o  goal  ...  length  met  like  prob  match
0      1.0      1.0  21.0   27.0   2.0  ...     2.0  1.0   7.0   6.0    0.0
1      1.0      2.0  21.0   22.0   2.0  ...     2.0  1.0   7.0   5.0    0.0
2      1.0      3.0  21.0   22.0   2.0  ...     2.0  1.0   7.0   NaN    1.0
3      1.0      4.0  21.0   23.0   2.0  ...     2.0  1.0   7.0   6.0    1.0
4      1.0      5.0  21.0   24.0   2.0  ...     2.0  1.0   6.0   6.0    1.0
...    ...      ...   ...    ...   ...  ...     ...  ...   ...   ...    ...
8373  22.0     18.0  25.0   26.0   1.0  ...     1.0  0.0   2.0   5.0    0.0
8374  22.0     19.0  25.0   24.0   1.0  ...     1.0  0.0   4.0   4.0    0.0
8375  22.0     20.0  25.0   29.0   1.0  ...     1.0  0.0   6.0   5.0    0.0
8376  22.0     21.0  25.0   22.0   1.0  ...     1.0  0.0   5.0   5.0    0.0
8377   NaN     22.0  25.0   22.0   1.0  ...     1.0  0.0   4.0   5.0    0.0

[8378 rows x 13 columns]


Descrição dos dados

In [ ]:
data.describe()

,id,partner,age,age_o,goal,date,go_out,int_corr,length,met,like,prob,match
count,8377.000000,8378.000000,8283.000000,8274.000000,8299.000000,8281.000000,8299.000000,8220.000000,7463.000000,8003.000000,8138.000000,8069.000000,8378.000000
mean,8.960248,8.963595,26.358928,26.364999,2.122063,5.006762,2.158091,0.196010,1.843495,0.494315,6.134087,5.207523,0.164717
std,5.491329,5.491068,3.566763,3.563648,1.407181,1.444531,1.105246,0.303539,0.975662,0.499999,1.841285,2.129565,0.370947
min,1.000000,1.000000,18.000000,18.000000,1.000000,1.000000,1.000000,-0.830000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,4.000000,24.000000,24.000000,1.000000,4.000000,1.000000,-0.020000,1.000000,0.000000,5.000000,4.000000,0.000000
50%,8.000000,8.000000,26.000000,26.000000,2.000000,5.000000,2.000000,0.210000,1.000000,0.000000,6.000000,5.000000,0.000000
75%,13.000000,13.000000,28.000000,28.000000,2.000000,6.000000,3.000000,0.430000,3.000000,1.000000,7.000000,7.000000,0.000000
max,22.000000,22.000000,55.000000,55.000000,6.000000,7.000000,7.000000,0.910000,3.000000,1.000000,10.000000,10.000000,1.000000


Substituição dos valores nulos pelo: valor inteiro da média dos seus valores em age e age_o, o int_corr é substituido pelo seu valor float com duas casas decimais e as variaveis categoricas sao substituidas pela sua moda. **substituir os valores quebrados da variavel like e probs para nao termos um modelo muito overthinngk comparar ma verdade**

In [ ]:
dataC=data.copy()


for col in dataC.columns:
  if(col!='id' and col!='partner'):
      if(col=='int_corr'):
         dataC[col]=dataC[col].replace(np.nan,round(dataC[col].mean(),2))
      if(col=='age' or col=='age_o'):
         dataC[col]=dataC[col].replace(np.nan,float(int(dataC[col].mean())))
      else:
         dataC[col]=dataC[col].replace(np.nan,float(int(dataC[col].mode())))
         


data=dataC.copy()



In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        8377 non-null   float64
 1   partner   8378 non-null   float64
 2   age       8378 non-null   float64
 3   age_o     8378 non-null   float64
 4   goal      8378 non-null   float64
 5   date      8378 non-null   float64
 6   go_out    8378 non-null   float64
 7   int_corr  8378 non-null   float64
 8   length    8378 non-null   float64
 9   met       8378 non-null   float64
 10  like      8378 non-null   float64
 11  prob      8378 non-null   float64
 12  match     8378 non-null   float64
dtypes: float64(13)
memory usage: 851.0 KB
None


Nova descrição dos dados

In [ ]:
data.describe()

,id,partner,age,age_o,goal,date,go_out,int_corr,length,met,like,prob,match
count,8377.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000
mean,8.960248,8.963595,26.354858,26.360468,2.111482,5.018262,2.156601,0.196085,1.751373,0.472189,6.158892,5.199869,0.164717
std,5.491329,5.491068,3.546684,3.541689,1.404722,1.440069,1.100129,0.300664,0.957688,0.499256,1.820457,2.090285,0.370947
min,1.000000,1.000000,18.000000,18.000000,1.000000,1.000000,1.000000,-0.830000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,4.000000,24.000000,24.000000,1.000000,4.000000,1.000000,-0.010000,1.000000,0.000000,5.000000,4.000000,0.000000
50%,8.000000,8.000000,26.000000,26.000000,2.000000,5.000000,2.000000,0.200000,1.000000,0.000000,6.000000,5.000000,0.000000
75%,13.000000,13.000000,28.000000,28.000000,2.000000,6.000000,3.000000,0.430000,3.000000,1.000000,7.000000,7.000000,0.000000
max,22.000000,22.000000,55.000000,55.000000,6.000000,7.000000,7.000000,0.910000,3.000000,1.000000,10.000000,10.000000,1.000000


# Analise dos dados (no R)




In [ ]:
#from google.colab import files
#data.to_csv('speedDating.csv',index = False, header = True)
#files.download('speedDating.csv')

# Preparação dos dados

Separação entre atributos e objetivos

In [ ]:
x=data.drop(columns=['id','partner','match'])
y=data['match']

30% dos dados para teste e 70% para treino

In [ ]:
from sklearn.model_selection import train_test_split
#random state para fixar uma divisão dos dados
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=9)



# Criando modelos CART

Sem pré prunning

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

model_SP=DecisionTreeClassifier(criterion='gini',random_state=9)
model_SP.fit(x_train,y_train)
#fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=300)
#tree.plot_tree(model_SP,feature_names=x.columns,class_names=["0","1"],filled=True)
#fig.savefig('arvore_SP.png')

predictions_cart_SP=model_SP.predict(x_test)

Analise CART prunning: verificar qual melhor altura da arvore

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
scorem=0
isc=0
precisionm=0
ip=0
recallm=0
ir=0

for i in range(1,1200):
  model_i=DecisionTreeClassifier(criterion='gini',max_depth=i,random_state=9)
  model_i.fit(x_train,y_train)
  predictions_cart_i=model_i.predict(x_test)
  score=accuracy_score(y_test, predictions_cart_i)
  precision=precision_score(y_test,predictions_cart_i,labels=[1,0])
  recall=recall_score(y_test,predictions_cart_i,labels=[1,0])
  
  if(score>scorem):
    scorem=score
    isc=i
  if(precision>precisionm):
    precisionm=precision
    ip=i
  if(recall>recallm):
    recallm=recall 
    ir=i
  
print("melhor score: altura ",isc," melhor precision: altura ",ip)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


melhor score: altura  3  melhor precision: altura  3


com pre prunnig

In [ ]:
model=DecisionTreeClassifier(criterion='gini',max_depth=18,random_state=9)
model.fit(x_train,y_train)
#fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=300)
#tree.plot_tree(model,feature_names=x.columns,class_names=["0","1"],filled=True)
#fig.savefig('arvore.png')

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=18, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=9, splitter='best')

Ver arvore

In [ ]:
predictions_cart=model.predict(x_test)

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train, y_train)
predictions_naive= clf.predict(x_test)



# Avaliação

Accurracy

In [ ]:
from sklearn.metrics import accuracy_score


score_cart_SP=accuracy_score(y_test, predictions_cart_SP)
print("accuracy cart s/p",score_cart_SP*100)

accuracy cart s/p 76.17342879872713


In [ ]:

score_cart=accuracy_score(y_test, predictions_cart)
print("accuracy cart c/p",score_cart*100)

accuracy cart c/p 76.41209228321401


In [ ]:
score_naive=accuracy_score(y_test, predictions_naive)
print("accuracy naive bayes",score_naive*100)

accuracy naive bayes 83.17422434367542


# dummy 

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf.fit(x_train, y_train)
DummyClassifier(random_state=9, strategy='most_frequent')
clf.score(x_test, y_test)*100

84.44709626093875

Matriz de confusão,precision e recall

CART s/prunning

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix_cart_SP=confusion_matrix(y_test, predictions_cart_SP,labels=[1,0])
confusion_matrix_cart_SP

array([[ 130,  261],
       [ 338, 1785]])

In [ ]:
from sklearn.metrics import precision_score, recall_score
print("precision ",precision_score(y_test,predictions_cart_SP,labels=[1,0])*100)
print("recall ",recall_score(y_test,predictions_cart_SP,labels=[1,0])*100)

precision  27.77777777777778
recall  33.248081841432224


CART c/ prunning

In [ ]:

confusion_matrix_cart=confusion_matrix(y_test, predictions_cart,labels=[1,0])
confusion_matrix_cart

array([[ 138,  253],
       [ 340, 1783]])

In [ ]:


print("precision ",precision_score(y_test,predictions_cart,labels=[1,0])*100)
print("recall ",recall_score(y_test,predictions_cart,labels=[1,0])*100)

precision  28.870292887029287
recall  35.294117647058826


Naive Bayes

In [ ]:
confusion_matrix_naive=confusion_matrix(y_test, predictions_naive,labels=[1,0])
confusion_matrix_naive

array([[  95,  296],
       [ 127, 1996]])

In [ ]:
print("precision ",precision_score(y_test,predictions_naive,labels=[1,0])*100)
print("recall ",recall_score(y_test,predictions_naive,labels=[1,0])*100)

precision  42.792792792792795
recall  24.296675191815854


# Score F1

In [ ]:
from sklearn.metrics import f1_score
print("score f1 cart s/p ",f1_score(y_test,predictions_cart_SP))
print("score f1 cart c/p ",f1_score(y_test,predictions_cart))
print("score f1 naive bayes ",f1_score(y_test,predictions_naive))

score f1 cart s/p  0.3026775320139697
score f1 cart c/p  0.31760644418872264
score f1 naive bayes  0.3099510603588907
